In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [10]:
#group_durationからの集計
filename = ['201401','201402','201403','201404','201405','201406','201407','201408','201409','201410','201411','201412','201501','201502','201503','201504']
for file in filename:
  df_data = pd.read_csv('/content/drive/MyDrive/研究/WiFiLog/wifidata/after/'+file+'/group_duration_'+file+'.csv')
  print(file)

  #ユニークユーザ数を数える
  unique_users = df_data['client'].drop_duplicates()
  unique_users = unique_users.reset_index(drop=True)
  unique_users.to_csv('/content/drive/MyDrive/研究/WiFiLog/wifidata/cluster/'+file+'/users_'+file+'.csv')

  #データのカウント
  count = np.zeros((len(unique_users),49)) #建物が48個、外部が49、1を引いて入れる
  duration = np.zeros((len(unique_users),49))
  index = 0

  for i in df_data.itertuples():
    if i.client == 'client': #途中でヘッダーが入ってしまった
      continue
    if unique_users[index] != i.client:
      index += 1
      print('{0} / {1} : User'.format(index, len(unique_users)))
    #print('From : {0}'.format(i[2]))
    count[index][int(i[2] -1)] += 1
    if int(i[2]) != 49:
      if float(i.duration) < 3600 * 3: #滞在時間を最大3時間とする
        duration[index][int(i[2] -1)] += float(i.duration)
      #print('Duration : {0}'.format(float(i.duration)))
    #if index > 10:
    #  break
  #print(count)
  #print(duration)

  np.savetxt('/content/drive/MyDrive/研究/WiFiLog/wifidata/cluster/'+file+'/count_'+file+'.csv',count,delimiter=',')
  np.savetxt('/content/drive/MyDrive/研究/WiFiLog/wifidata/cluster/'+file+'/duration_'+file+'_3h.csv',duration,delimiter=',')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Streaming output truncated to the last 5000 lines.
27418 / 32418 : User
27419 / 32418 : User
27420 / 32418 : User
27421 / 32418 : User
27422 / 32418 : User
27423 / 32418 : User
27424 / 32418 : User
27425 / 32418 : User
27426 / 32418 : User
27427 / 32418 : User
27428 / 32418 : User
27429 / 32418 : User
27430 / 32418 : User
27431 / 32418 : User
27432 / 32418 : User
27433 / 32418 : User
27434 / 32418 : User
27435 / 32418 : User
27436 / 32418 : User
27437 / 32418 : User
27438 / 32418 : User
27439 / 32418 : User
27440 / 32418 : User
27441 / 32418 : User
27442 / 32418 : User
27443 / 32418 : User
27444 / 32418 : User
27445 / 32418 : User
27446 / 32418 : User
27447 / 32418 : User
27448 / 32418 : User
27449 / 32418 : User
27450 / 32418 : User
27451 / 32418 : User
27452 / 32418 : User
27453 / 32418 : User
27454 / 32418 : User
27455 / 32418 : User
27456 / 32418 : User
27457 / 32418 : User
27458 / 32418 : User
27459 / 32418 : User
27460 / 32418 : User
27461 / 32418 : User
27462 / 32418 : User
2746

In [18]:
#Kmeansと集約
for file in filename:
  print(file)
  df_count = pd.read_csv('/content/drive/MyDrive/研究/WiFiLog/wifidata/cluster/'+file+'/count_'+file+'.csv',header=None)
  df_duration = pd.read_csv('/content/drive/MyDrive/研究/WiFiLog/wifidata/cluster/'+file+'/duration_'+file+'_3h.csv',header=None)
  df_duration = df_duration / df_count
  df_duration = df_duration.fillna(0)
  df_users = pd.read_csv('/content/drive/MyDrive/研究/WiFiLog/wifidata/cluster/'+file+'/users_'+file+'.csv',index_col=0)

  df_data = pd.concat([df_users, df_count, df_duration], axis=1)
  from sklearn.cluster import KMeans
  kmodel = KMeans(n_clusters= 10, random_state= 111)
  kmodel.fit(df_data.iloc[:,1:-1])
  print(kmodel.labels_)

  df_data['kmeans'] = kmodel.labels_
  df_data.to_csv('/content/drive/MyDrive/研究/WiFiLog/wifidata/cluster/'+file+'/concat_'+file+'_3h.csv', index = False)

201401
[2 9 7 ... 9 9 1]
201402
[1 2 1 ... 0 6 1]
201403
[0 4 0 ... 0 0 9]
201404
[0 0 3 ... 0 7 3]
201405
[1 0 0 ... 2 1 0]
201406
[2 4 2 ... 2 2 2]
201407
[2 2 2 ... 2 2 2]
201408
[1 1 1 ... 1 1 1]
201409
[1 4 6 ... 1 8 6]
201410
[0 0 0 ... 0 0 0]
201411
[0 0 5 ... 0 5 0]
201412
[1 1 1 ... 1 1 7]
201501
[1 0 1 ... 0 0 1]
201502
[1 1 1 ... 1 8 1]
201503
[1 1 0 ... 1 1 1]
201504
[5 0 2 ... 8 2 2]


In [19]:
#ユニークユーザ数を数える
#unique_users = df_data['client'].drop_duplicates()
#unique_users = unique_users.reset_index(drop=True)
#unique_users.to_csv('/content/drive/MyDrive/研究/WiFiLog/wifidata/cluster/201401/users_201401.csv')
#df_data[df_data['client'] == 'f4c8b3e34e87788c1170747db471efcf7ff6603c']

In [20]:
'''
count = np.zeros((len(unique_users),49)) #建物が48個、外部が49、1を引いて入れる
duration = np.zeros((len(unique_users),49))
index = 0

for i in df_data.itertuples():
  if i.client == 'client': #途中でヘッダーが入ってしまった
    continue
  if unique_users[index] != i.client:
    index += 1
    print('{0} / {1} : User'.format(index, len(unique_users)))
  #print('From : {0}'.format(i[2]))
  count[index][int(i[2] -1)] += 1
  if int(i[2]) != 49:
    if float(i.duration) < 3600 * 3: #滞在時間を最大3時間とする
      duration[index][int(i[2] -1)] += float(i.duration)
    #print('Duration : {0}'.format(float(i.duration)))
  #if index > 10:
  #  break
#print(count)
#print(duration)

np.savetxt('/content/drive/MyDrive/研究/WiFiLog/wifidata/cluster/201401/count_201401.csv',count,delimiter=',')
np.savetxt('/content/drive/MyDrive/研究/WiFiLog/wifidata/cluster/201401/duration_201401_3h.csv',duration,delimiter=',')
'''

"\ncount = np.zeros((len(unique_users),49)) #建物が48個、外部が49、1を引いて入れる\nduration = np.zeros((len(unique_users),49))\nindex = 0\n\nfor i in df_data.itertuples():\n  if i.client == 'client': #途中でヘッダーが入ってしまった\n    continue\n  if unique_users[index] != i.client:\n    index += 1\n    print('{0} / {1} : User'.format(index, len(unique_users)))\n  #print('From : {0}'.format(i[2]))\n  count[index][int(i[2] -1)] += 1\n  if int(i[2]) != 49:\n    if float(i.duration) < 3600 * 3: #滞在時間を最大3時間とする\n      duration[index][int(i[2] -1)] += float(i.duration)\n    #print('Duration : {0}'.format(float(i.duration)))\n  #if index > 10:\n  #  break\n#print(count)\n#print(duration)\n\nnp.savetxt('/content/drive/MyDrive/研究/WiFiLog/wifidata/cluster/201401/count_201401.csv',count,delimiter=',')\nnp.savetxt('/content/drive/MyDrive/研究/WiFiLog/wifidata/cluster/201401/duration_201401_3h.csv',duration,delimiter=',')\n"